Importing all the libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()


Loading the data

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Sepsis Prediction/cleaned_data.csv')
df

,Hour,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,Age,Gender,Unit1,HospAdmTime,ICULOS,SepsisLabel,Patient_ID
0,0,83.5,98.0,37.0,121.0,80.0,62.0,18.0,68.54,0,0.0,0.0,1,0,17072
1,1,65.0,100.0,37.0,121.0,72.0,62.0,16.5,68.54,0,0.0,0.0,2,0,17072
2,2,78.0,100.0,37.0,121.0,42.5,62.0,18.0,68.54,0,0.0,0.0,3,0,17072
3,3,73.0,100.0,37.0,121.0,80.0,62.0,17.0,68.54,0,0.0,0.0,4,0,17072
4,4,70.0,100.0,37.0,129.0,74.0,69.0,14.0,68.54,0,0.0,0.0,5,0,17072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1552205,21,83.0,99.0,37.0,121.0,77.0,54.0,22.0,88.00,0,1.0,0.0,22,0,113911
1552206,22,80.0,92.0,37.0,102.0,73.0,51.0,24.0,88.00,0,1.0,0.0,23,0,113911
1552207,23,95.0,97.0,36.7,128.5,83.0,58.5,25.0,88.00,0,1.0,0.0,24,0,113911
1552208,24,104.0,99.0,37.0,127.0,85.0,59.0,24.0,88.00,0,1.0,0.0,25,0,113911


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1552210 entries, 0 to 1552209
Data columns (total 15 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Hour         1552210 non-null  int64  
 1   HR           1552210 non-null  float64
 2   O2Sat        1552210 non-null  float64
 3   Temp         1552210 non-null  float64
 4   SBP          1552210 non-null  float64
 5   MAP          1552210 non-null  float64
 6   DBP          1552210 non-null  float64
 7   Resp         1552210 non-null  float64
 8   Age          1552210 non-null  float64
 9   Gender       1552210 non-null  int64  
 10  Unit1        1552210 non-null  float64
 11  HospAdmTime  1552210 non-null  float64
 12  ICULOS       1552210 non-null  int64  
 13  SepsisLabel  1552210 non-null  int64  
 14  Patient_ID   1552210 non-null  int64  
dtypes: float64(10), int64(5)
memory usage: 177.6 MB


Aggregrating the data

In [ ]:
sepsis_onset = df[df['SepsisLabel'] == 1].groupby('Patient_ID')['Hour'].min().reset_index()
sepsis_onset.rename(columns={'Hour': 'SepsisOnsetHour'}, inplace=True)

df = df.merge(sepsis_onset, on='Patient_ID', how='left')

df_filtered = df[
    (df['SepsisOnsetHour'].isna()) |
    (df['Hour'] <= df['SepsisOnsetHour'])
].copy()


df_filtered.drop(columns=['SepsisOnsetHour'], inplace=True)

agg_funcs = {
    'HR': ['mean', 'min', 'max', 'std'],
    'O2Sat': ['mean', 'min', 'max', 'std'],
    'Temp': ['mean', 'min', 'max', 'std'],
    'SBP': ['mean', 'min', 'max', 'std'],
    'MAP': ['mean', 'min', 'max', 'std'],
    'DBP': ['mean', 'min', 'max', 'std'],
    'Resp': ['mean', 'min', 'max', 'std'],
    'Age': 'first',
    'Gender': 'first',
    'Unit1': 'first',
    'HospAdmTime': 'first',
    'ICULOS': 'max',
    'SepsisLabel': 'max'

df_agg = df_filtered.groupby('Patient_ID').agg(agg_funcs)

df_agg.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_agg.columns]

df_agg.reset_index(inplace=True)

df_agg.head()


,Patient_ID,HR_mean,HR_min,HR_max,HR_std,O2Sat_mean,O2Sat_min,O2Sat_max,O2Sat_std,Temp_mean,...,Resp_mean,Resp_min,Resp_max,Resp_std,Age_first,Gender_first,Unit1_first,HospAdmTime_first,ICULOS_max,SepsisLabel_max
0,1,99.898148,76.0,117.0,10.551035,92.685185,85.0,100.0,4.032061,36.958889,...,24.314815,17.0,32.0,4.340785,83.14,0,0.0,0.0,54,0
1,2,61.934783,54.0,94.0,9.242078,97.043478,94.0,100.0,2.099313,36.782174,...,14.891304,9.0,27.0,4.466940,75.91,0,0.0,0.0,23,0
2,3,79.854167,68.0,93.0,6.565511,95.645833,91.0,99.0,1.738054,37.203125,...,25.156250,17.0,40.0,4.513757,45.82,0,1.0,0.0,48,0
3,4,101.137931,83.5,113.0,7.820413,98.189655,95.5,100.0,1.397790,36.887241,...,18.793103,14.0,26.0,3.546923,65.71,0,0.0,0.0,29,0
4,5,78.708333,61.0,88.0,7.184375,97.781250,96.0,99.0,0.544862,36.998542,...,17.343750,14.0,21.0,1.388234,28.09,1,1.0,0.0,49,0


In [ ]:
df_agg

,Patient_ID,HR_mean,HR_min,HR_max,HR_std,O2Sat_mean,O2Sat_min,O2Sat_max,O2Sat_std,Temp_mean,...,Resp_mean,Resp_min,Resp_max,Resp_std,Age_first,Gender_first,Unit1_first,HospAdmTime_first,ICULOS_max,SepsisLabel_max
0,1,99.898148,76.0,117.0,10.551035,92.685185,85.0,100.0,4.032061,36.958889,...,24.314815,17.0,32.0,4.340785,83.14,0,0.0,0.0,54,0
1,2,61.934783,54.0,94.0,9.242078,97.043478,94.0,100.0,2.099313,36.782174,...,14.891304,9.0,27.0,4.466940,75.91,0,0.0,0.0,23,0
2,3,79.854167,68.0,93.0,6.565511,95.645833,91.0,99.0,1.738054,37.203125,...,25.156250,17.0,40.0,4.513757,45.82,0,1.0,0.0,48,0
3,4,101.137931,83.5,113.0,7.820413,98.189655,95.5,100.0,1.397790,36.887241,...,18.793103,14.0,26.0,3.546923,65.71,0,0.0,0.0,29,0
4,5,78.708333,61.0,88.0,7.184375,97.781250,96.0,99.0,0.544862,36.998542,...,17.343750,14.0,21.0,1.388234,28.09,1,1.0,0.0,49,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40331,119996,89.208333,69.0,124.0,12.584960,98.083333,95.0,100.0,0.985714,36.875000,...,18.833333,16.0,23.0,1.575522,84.00,0,0.0,0.0,48,0
40332,119997,61.960000,44.0,83.5,11.433613,96.880000,89.0,100.0,2.255918,36.976000,...,19.240000,14.0,26.0,3.205204,30.00,1,0.0,0.0,25,0
40333,119998,80.673469,57.0,103.0,8.967854,96.469388,88.0,100.0,3.670559,36.934694,...,20.000000,17.0,26.0,2.148158,60.00,0,1.0,0.0,49,0
40334,119999,93.250000,83.5,106.0,6.014237,93.250000,83.0,100.0,4.050666,37.155000,...,21.300000,17.0,27.0,2.319256,84.00,0,1.0,0.0,20,0


Changing Column Names

In [ ]:
df_agg.rename(columns={
    'Age_first': 'Age',
    'Gender_first': 'Gender',
    'Unit1_first': 'Unit1',
    'HospAdmTime_first': 'HospAdmTime',
    'ICULOS_max': 'ICULOS',
    'SepsisLabel_max': 'SepsisLabel'
}, inplace=True)


In [ ]:
std_cols = [col for col in df_agg.columns if col.endswith('_std')]
df_agg[std_cols] = df_agg[std_cols].fillna(0)


In [ ]:
df_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40336 entries, 0 to 40335
Data columns (total 35 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Patient_ID   40336 non-null  int64  
 1   HR_mean      40336 non-null  float64
 2   HR_min       40336 non-null  float64
 3   HR_max       40336 non-null  float64
 4   HR_std       40336 non-null  float64
 5   O2Sat_mean   40336 non-null  float64
 6   O2Sat_min    40336 non-null  float64
 7   O2Sat_max    40336 non-null  float64
 8   O2Sat_std    40336 non-null  float64
 9   Temp_mean    40336 non-null  float64
 10  Temp_min     40336 non-null  float64
 11  Temp_max     40336 non-null  float64
 12  Temp_std     40336 non-null  float64
 13  SBP_mean     40336 non-null  float64
 14  SBP_min      40336 non-null  float64
 15  SBP_max      40336 non-null  float64
 16  SBP_std      40336 non-null  float64
 17  MAP_mean     40336 non-null  float64
 18  MAP_min      40336 non-null  float64
 19  MAP_

Saving Aggregated Data

In [ ]:
df_agg.to_csv('/content/drive/MyDrive/Sepsis Prediction/aggregated_data1.csv', index=False)

In [ ]:
df_agg['SepsisLabel'].value_counts()


,count
SepsisLabel,
0,37404
1,2932
